# CNN_MNIST with Visdom

In [3]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [4]:
import visdom

vis= visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

In [7]:
def loss_tracker(loss_plt, loss_value, num):
    vis.line(X=num, Y=loss_value, win = loss_plt, update ='append')

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# gpu나 cpu를 통해 연산
# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [9]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [10]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [11]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [12]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [13]:
# CNN 모델 정의
model = CNN().to(device)

In [14]:
criterion = torch.nn.CrossEntropyLoss().to(device)    
# 비용 함수에 소프트맥스 함수(원-핫 인코딩도) 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### make plot

In [17]:
loss_plt = vis.line(Y= torch.Tensor(1).zero_(), opts=dict(tile='loss_tracker', legend = ['loss'], showlegend = True))

In [18]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))
#배치의 수: iterator = len(data_loader)

총 배치의 수 : 600


In [20]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: 
        # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))

[Epoch:    1] cost = 0.0621448644
[Epoch:    2] cost = 0.0449068099
[Epoch:    3] cost = 0.0354769863
[Epoch:    4] cost = 0.0289367419
[Epoch:    5] cost = 0.0248863045
[Epoch:    6] cost = 0.0206354484
[Epoch:    7] cost = 0.018088242
[Epoch:    8] cost = 0.0151171302
[Epoch:    9] cost = 0.0123944618
[Epoch:   10] cost = 0.0102480575
[Epoch:   11] cost = 0.00983714592
[Epoch:   12] cost = 0.00913958345
[Epoch:   13] cost = 0.0066404324
[Epoch:   14] cost = 0.00555599341
[Epoch:   15] cost = 0.0057003987
